In [1]:
import os
import boto3
from config import *
import base64

In [2]:
print(boto3.__version__)

1.37.4


In [3]:
bedrock_agent = boto3.Session().client('bedrock-agent')

In [4]:
path = "/home/sagemaker-user/.cache/kagglehub/datasets/hsankesara/flickr-image-dataset/versions/1/flickr30k_images/flickr30k_images"
print(path)

/home/sagemaker-user/.cache/kagglehub/datasets/hsankesara/flickr-image-dataset/versions/1/flickr30k_images/flickr30k_images


In [5]:
counter = 0
for filename in os.listdir(path):
    if counter %1000 == 0:
        print(f"{counter}th == {filename}")
    counter +=1

0th == 1000092795.jpg
1000th == 131624221.jpg
2000th == 1572286502.jpg
3000th == 2037264035.jpg
4000th == 2201202388.jpg
5000th == 233242340.jpg
6000th == 2450486758.jpg
7000th == 2573009825.jpg
8000th == 2705102979.jpg
9000th == 2834032170.jpg
10000th == 2954929479.jpg
11000th == 3085592876.jpg
12000th == 3201065069.jpg
13000th == 3315323307.jpg
14000th == 3424424006.jpg
15000th == 3533145793.jpg
16000th == 3643974707.jpg
17000th == 378453580.jpg
18000th == 4011411161.jpg
19000th == 4273698621.jpg
20000th == 4464227525.jpg
21000th == 4575727965.jpg
22000th == 4676717700.jpg
23000th == 4752984291.jpg
24000th == 4830307454.jpg
25000th == 491484504.jpg
26000th == 503357436.jpg
27000th == 5447844412.jpg
28000th == 6041156608.jpg
29000th == 67491882.jpg
30000th == 7283335868.jpg
31000th == 80474519.jpg


In [6]:
# If you're using PIL/Pillow Image object
import io
import base64

In [8]:
## updated this section
def image_to_bytes(image_path):
    with open(image_path, "rb") as image_file:
        # Read the file content
        binary_data = image_file.read()
        # Encode the binary data to base64
        base64_encoded = base64.b64encode(binary_data)
        # Convert bytes to string
        base64_string = base64_encoded.decode('utf-8')     #  <---- adding this step
    return {'data': base64_string}

In [9]:
## example usage
image_path = path + '/1314231418.jpg'
image_bytes = image_to_bytes(image_path)
image_bytes_data = image_bytes['data']

In [10]:
print(type(image_bytes['data']))

<class 'str'>


In [11]:
import random
import string
import time

In [12]:
def generate_random_code():
    # Generate 4 random characters (uppercase letters)
    characters = ''.join(random.choices(string.ascii_uppercase, k=4))
    
    # Generate 12 digit number combining timestamp and random numbers
    timestamp = str(int(time.time()))
    random_digits = ''.join(random.choices(string.digits, k=6))
    number = (timestamp + random_digits)[-12:]  # Ensure 12 digits
    
    return characters, number

In [15]:
# now we will test uploading a single image
def upload_image(image_path):
    image_bytes = image_to_bytes(image_path)
    image_bytes_data = image_bytes['data']
    
    random_code = generate_random_code()
    documentIdentifier = str(random_code[0]) + str(random_code[1]) 
    document = {
        'content': {
            'custom': {
                'customDocumentIdentifier': {
                    'id': documentIdentifier
                },
                'inlineContent': {
                    'byteContent': {
                        'data': image_bytes_data,
                        'mimeType': 'image/jpeg'
                    },
                    'type': 'BYTE'
                },
                'sourceType': 'IN_LINE'
            },
            'dataSourceType': 'CUSTOM',
        }
    }
    documents = [document]

    response = bedrock_agent.ingest_knowledge_base_documents(
        dataSourceId = data_source_id, documents = documents, knowledgeBaseId = knowledgebase_id
    )
    return response

In [16]:
### inline upload
counter = 0
for filename in os.listdir(path):
    if counter == 3:
        break
    image_path = path + "/" + filename
    upload_response = upload_image(image_path)
    timer = 0
    print(" --- Updload response")
    print(upload_response)
    print(" --- ")
    while timer < 10:
        time.sleep(1)
        timer +=1
    print(image_path)
    counter +=1

 --- Updload response
{'ResponseMetadata': {'RequestId': 'deef53fd-b0c2-4119-ac17-3a1b4ebb1d60', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Tue, 04 Mar 2025 16:55:39 GMT', 'content-type': 'application/json', 'content-length': '223', 'connection': 'keep-alive', 'x-amzn-requestid': 'deef53fd-b0c2-4119-ac17-3a1b4ebb1d60', 'x-amz-apigw-id': 'G6SFiG0bIAMEH-A=', 'x-amzn-trace-id': 'Root=1-67c73089-1be35aae430539205cc773f6'}, 'RetryAttempts': 0}, 'documentDetails': [{'dataSourceId': 'E3NRB1GDE0', 'identifier': {'custom': {'id': 'ZRVX107337149534'}, 'dataSourceType': 'CUSTOM'}, 'knowledgeBaseId': '5WYB0DMRMN', 'status': 'STARTING', 'updatedAt': datetime.datetime(2025, 3, 4, 16, 55, 39, 362377, tzinfo=tzlocal())}]}
 --- 
/home/sagemaker-user/.cache/kagglehub/datasets/hsankesara/flickr-image-dataset/versions/1/flickr30k_images/flickr30k_images/1000092795.jpg
 --- Updload response
{'ResponseMetadata': {'RequestId': 'be7bc7d3-245f-47eb-82ca-360edeba1097', 'HTTPStatusCode': 202, 'HTTPHeaders':

I was getting logs of this kind
```
{
    "event_timestamp": 1740509325142,
    "event": {
        "document_location": {
            "customDocument_location": {
                "id": "***"
            },
            "type": "CUSTOM"
        },
        "request_id": "********",
        "data_source_id": "********",
        "status_reasons": [
            "Resource not processed due to a service exception. Please reach out to Amazon Bedrock technical support team for further assistance, or contact technical support at aws.amazon.com/contact-us/."
        ],
        "knowledge_base_arn": "arn:aws:bedrock:us-east-1:**********:knowledge-base/********",
        "status": "RESOURCE_IGNORED"
    },
    "event_version": "1.0",
    "event_type": "IngestKnowledgeBaseDocuments.ResourceStatusChanged",
    "level": "WARN"
}
```

now I am getting this error:

```
{
    "event_timestamp": 1741107340497,
    "event": {
        "document_location": {
            "customDocument_location": {
                "id": "**********"
            },
            "type": "CUSTOM"
        },
        "request_id": "*********",
        "data_source_id": "*******",
        "status_reasons": [
            "The server encountered an internal error while processing the request."
        ],
        "knowledge_base_arn": "arn:aws:bedrock:us-east-1:794038231401:knowledge-base/5WYB0DMRMN",
        "status": "RESOURCE_IGNORED"
    },
    "event_version": "1.0",
    "event_type": "IngestKnowledgeBaseDocuments.ResourceStatusChanged",
    "level": "ERROR"
}
```

uploading 6 GB of data will take an excesive amount of time and resources to test. For instance, when I first attempted to upload a 17 MB pdf file (the bedrock user guide actually) which contains more than 2500 pages, It was taking longer than 8 hours and I had to stop the test for that reason. Hence uploading 6 GB of data will take 118 days.

In [15]:
s3_location = 's3://mainbucketrockhight5461/test/knowledge-bases/direct_KB_upload/1image/371897.jpg'

In [16]:
# I will attempt to upload the images using s3 direct upload to custom source
def s3_upload_image(s3_location):
    
    random_code = generate_random_code()
    documentIdentifier = str(random_code[0]) + str(random_code[1]) 
    document = { 
        'content':{
            'dataSourceType': 'CUSTOM',
            'custom':{
                'customDocumentIdentifier': {
                    'id': documentIdentifier
                },
                's3Location': {
                    'bucketOwnerAccountId': bucket_owner_account,
                    'uri': s3_location
                },
                'sourceType': 'S3_LOCATION'
            }
        }
    }
    print(documentIdentifier)
    
    documents = [document]

    response = bedrock_agent.ingest_knowledge_base_documents(
        dataSourceId = data_source_id, documents = documents, knowledgeBaseId = knowledgebase_id
    )
    return response

In [17]:
s3_upload_response = s3_upload_image(s3_location)

DZAW035522177445


In [18]:
print(s3_upload_response)

{'ResponseMetadata': {'RequestId': '7b6d842a-524b-4f0a-89a8-7aec2c7523fa', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Mon, 03 Mar 2025 20:58:44 GMT', 'content-type': 'application/json', 'content-length': '223', 'connection': 'keep-alive', 'x-amzn-requestid': '7b6d842a-524b-4f0a-89a8-7aec2c7523fa', 'x-amz-apigw-id': 'G3iwfEN-IAMETmw=', 'x-amzn-trace-id': 'Root=1-67c61802-60421cfa7f48efec383fcb90'}, 'RetryAttempts': 0}, 'documentDetails': [{'dataSourceId': 'E3NRB1GDE0', 'identifier': {'custom': {'id': 'DZAW035522177445'}, 'dataSourceType': 'CUSTOM'}, 'knowledgeBaseId': '5WYB0DMRMN', 'status': 'STARTING', 'updatedAt': datetime.datetime(2025, 3, 3, 20, 58, 44, 237544, tzinfo=tzlocal())}]}


In [19]:
# the request failed, I will test now using a json file
s3_response = s3_upload_image('s3://mainbucketrockhight5461/test/documentation/bedrock-user-guide.pdf.metadata.json')

DAAE035643656445


In [20]:
print(s3_response)

{'ResponseMetadata': {'RequestId': 'cb6a6faa-d6fe-4bed-936f-5b38200095f4', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Mon, 03 Mar 2025 21:00:45 GMT', 'content-type': 'application/json', 'content-length': '223', 'connection': 'keep-alive', 'x-amzn-requestid': 'cb6a6faa-d6fe-4bed-936f-5b38200095f4', 'x-amz-apigw-id': 'G3jDXGFGIAMEQpw=', 'x-amzn-trace-id': 'Root=1-67c6187b-29acb6a27f7996755d87598e'}, 'RetryAttempts': 0}, 'documentDetails': [{'dataSourceId': 'E3NRB1GDE0', 'identifier': {'custom': {'id': 'DAAE035643656445'}, 'dataSourceType': 'CUSTOM'}, 'knowledgeBaseId': '5WYB0DMRMN', 'status': 'STARTING', 'updatedAt': datetime.datetime(2025, 3, 3, 21, 0, 45, 212667, tzinfo=tzlocal())}]}


In [22]:
## I think the ingestions might be failing because we already reached the limit actually. So I will attempt with a new knowledge base

In [21]:
# I will attempt to upload the images using s3 direct upload to custom source
def s3_upload_image_nds(s3_location, data_source_id):
    
    random_code = generate_random_code()
    documentIdentifier = str(random_code[0]) + str(random_code[1]) 
    document = { 
        'content':{
            'dataSourceType': 'CUSTOM',
            'custom':{
                'customDocumentIdentifier': {
                    'id': documentIdentifier
                },
                's3Location': {
                    'bucketOwnerAccountId': bucket_owner_account,
                    'uri': s3_location
                },
                'sourceType': 'S3_LOCATION'
            }
        }
    }
    
    documents = [document]

    response = bedrock_agent.ingest_knowledge_base_documents(
        dataSourceId = data_source_id, documents = documents, knowledgeBaseId = knowledgebase_id
    )
    return response

# now we will test uploading a single image
def upload_image_nds(image_path, data_source_id):
    image_bytes = pil_image_to_bytes(image_path)
    image_bytes_data = image_bytes['data']
    
    random_code = generate_random_code()
    documentIdentifier = str(random_code[0]) + str(random_code[1]) 
    document = {
        'content': {
            'custom': {
                'customDocumentIdentifier': {
                    'id': documentIdentifier
                },
                'inlineContent': {
                    'byteContent': {
                        'data': image_bytes_data,
                        'mimeType': 'image/jpeg'
                    },
                    'type': 'BYTE'
                },
                'sourceType': 'IN_LINE'
            },
            'dataSourceType': 'CUSTOM',
        }
    }
    documents = [document]

    response = bedrock_agent.ingest_knowledge_base_documents(
        dataSourceId = data_source_id, documents = documents, knowledgeBaseId = knowledgebase_id
    )
    return response

In [22]:
counter = 0
for filename in os.listdir(path):
    if counter == 10:
        break
    image_path = path + "/" + filename
    upload_response = upload_image_nds(image_path, new_data_source)
    print(upload_response)
    timer = 0
    time.sleep(5)
    print(image_path)
    counter +=1

{'ResponseMetadata': {'RequestId': '1d6d1a88-e035-4468-8a48-b26cc06ae157', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Mon, 03 Mar 2025 21:03:44 GMT', 'content-type': 'application/json', 'content-length': '223', 'connection': 'keep-alive', 'x-amzn-requestid': '1d6d1a88-e035-4468-8a48-b26cc06ae157', 'x-amz-apigw-id': 'G3jfTE8mIAMETyg=', 'x-amzn-trace-id': 'Root=1-67c6192e-08b06b7a60203bb17c122551'}, 'RetryAttempts': 0}, 'documentDetails': [{'dataSourceId': 'G7YKZ4SMPY', 'identifier': {'custom': {'id': 'RJLK035822641249'}, 'dataSourceType': 'CUSTOM'}, 'knowledgeBaseId': '5WYB0DMRMN', 'status': 'STARTING', 'updatedAt': datetime.datetime(2025, 3, 3, 21, 3, 44, 321430, tzinfo=tzlocal())}]}
/home/sagemaker-user/.cache/kagglehub/datasets/hsankesara/flickr-image-dataset/versions/1/flickr30k_images/flickr30k_images/1000092795.jpg
{'ResponseMetadata': {'RequestId': 'd65936c5-272e-4f5d-ab97-3bbb666cf67c', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Mon, 03 Mar 2025 21:03:52 GMT', 'content

In [ ]:
# uploads are failing manually apparently so I will attempt to upload the image using s3 source

In [39]:
counter = 0
for filename in os.listdir(path):
    if counter == 10:
        break
    counter +=1
    image_path = path + "/" + filename
    print(image_path)

/home/sagemaker-user/.cache/kagglehub/datasets/hsankesara/flickr-image-dataset/versions/1/flickr30k_images/flickr30k_images/1000092795.jpg
/home/sagemaker-user/.cache/kagglehub/datasets/hsankesara/flickr-image-dataset/versions/1/flickr30k_images/flickr30k_images/10002456.jpg
/home/sagemaker-user/.cache/kagglehub/datasets/hsankesara/flickr-image-dataset/versions/1/flickr30k_images/flickr30k_images/1000268201.jpg
/home/sagemaker-user/.cache/kagglehub/datasets/hsankesara/flickr-image-dataset/versions/1/flickr30k_images/flickr30k_images/1000344755.jpg
/home/sagemaker-user/.cache/kagglehub/datasets/hsankesara/flickr-image-dataset/versions/1/flickr30k_images/flickr30k_images/1000366164.jpg
/home/sagemaker-user/.cache/kagglehub/datasets/hsankesara/flickr-image-dataset/versions/1/flickr30k_images/flickr30k_images/1000523639.jpg
/home/sagemaker-user/.cache/kagglehub/datasets/hsankesara/flickr-image-dataset/versions/1/flickr30k_images/flickr30k_images/1000919630.jpg
/home/sagemaker-user/.cache/k

In [43]:
s3_response = s3_upload_image_nds('s3://mainbucketrockhight5461/test/knowledge-bases/documentacion.txt', new_data_source)

In [45]:
# now I will attempt to upload an image
s3_response = s3_upload_image_nds('s3://mainbucketrockhight5461/test/knowledge-bases/direct_KB_upload/1image/371897.jpg', new_data_source)

In [46]:
print(s3_response)

{'ResponseMetadata': {'RequestId': '95867b3c-4f0c-4bcb-9d12-5e75479e2ea5', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Fri, 28 Feb 2025 22:19:41 GMT', 'content-type': 'application/json', 'content-length': '223', 'connection': 'keep-alive', 'x-amzn-requestid': '95867b3c-4f0c-4bcb-9d12-5e75479e2ea5', 'x-amz-apigw-id': 'Gt1zUE0EIAMEX8g=', 'x-amzn-trace-id': 'Root=1-67c2367b-2dac364d71f4e1484d644d79'}, 'RetryAttempts': 0}, 'documentDetails': [{'dataSourceId': 'G7YKZ4SMPY', 'identifier': {'custom': {'id': 'QKHT781179155434'}, 'dataSourceType': 'CUSTOM'}, 'knowledgeBaseId': '5WYB0DMRMN', 'status': 'STARTING', 'updatedAt': datetime.datetime(2025, 2, 28, 22, 19, 41, 264175, tzinfo=tzlocal())}]}


In [28]:
# now we will attempt to upload txt files and json files inline

def text_to_bytes(file_path, encoding='utf-8'):
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content.encode(encoding)
    except Exception as e:
        print(f"Error reading file: {e}")
        return None

def upload_txt_custom(text_path, data_source_id, mime_type):
    bytes_data = text_to_bytes(text_path)
    
    random_code = generate_random_code()
    documentIdentifier = str(random_code[0]) + str(random_code[1]) 
    document = {
        'content': {
            'custom': {
                'customDocumentIdentifier': {
                    'id': documentIdentifier
                },
                'inlineContent': {
                    'byteContent': {
                        'data': bytes_data,
                        'mimeType': mime_type
                    },
                    'type': 'BYTE'
                },
                'sourceType': 'IN_LINE'
            },
            'dataSourceType': 'CUSTOM',
        }
    }
    documents = [document]

    response = bedrock_agent.ingest_knowledge_base_documents(
        dataSourceId = data_source_id, documents = documents, knowledgeBaseId = knowledgebase_id
    )
    return response

In [29]:
mime_type = "text/plain"

In [30]:
file = "/home/sagemaker-user/user-default-efs/documents/mygithub/aws-samples/bedrock/miscelanious/test.txt"

In [31]:
file_response = upload_txt_custom(file, new_data_source, mime_type)

In [32]:
print(file_response)

{'ResponseMetadata': {'RequestId': '3c220139-367c-4b36-9edf-00563d253283', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Fri, 28 Feb 2025 21:56:31 GMT', 'content-type': 'application/json', 'content-length': '223', 'connection': 'keep-alive', 'x-amzn-requestid': '3c220139-367c-4b36-9edf-00563d253283', 'x-amz-apigw-id': 'GtyaIEsmoAMERhA=', 'x-amzn-trace-id': 'Root=1-67c2310d-4bb6a5ce456f2ca70d3c9b1b'}, 'RetryAttempts': 0}, 'documentDetails': [{'dataSourceId': 'G7YKZ4SMPY', 'identifier': {'custom': {'id': 'SQVZ779789923544'}, 'dataSourceType': 'CUSTOM'}, 'knowledgeBaseId': '5WYB0DMRMN', 'status': 'STARTING', 'updatedAt': datetime.datetime(2025, 2, 28, 21, 56, 30, 999715, tzinfo=tzlocal())}]}
